In [ ]:
!pip install transformers
!pip install eli5

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/GPT

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/GPT


In [ ]:
# Import generic wrappers
from transformers import AutoModel, AutoTokenizer , AutoModelForSequenceClassification, pipeline
import numpy as np
import pandas as pd

### Provide the input file name
input_text_file = "DagPap22_Corpus.csv"

# Define the model repo
model_name = "roberta-base-openai-detector"

# Download pytorch model
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
df_combined = pd.read_csv(input_text_file)
df_delta=pd.DataFrame(columns=['label', 'score'])
for txt in df_combined['text']:
  pred_pipeline  = pipeline("text-classification", model="roberta-base-openai-detector")
  # print(len(my_pipeline.tokenizer.tokenize(input_text)))
  df_delta=df_delta.append(pred_pipeline(txt, truncation=True))

In [ ]:
print(df_delta.shape)
print(df_combined.shape)

(232, 2)
(232, 6)


In [ ]:
df_combined.reset_index(drop=True, inplace=True)
df_delta.reset_index(drop=True, inplace=True)
df_final= pd.concat([df_combined, df_delta], axis=1)
df_final['label'] = df_final['label'].map({'Fake':'chatgpt', 'Real':'human'})

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print(classification_report(df_combined['source'], df_final['label']))
print(confusion_matrix(df_combined['source'], df_final['label']))

In [ ]:
df_final.to_csv("DagPap22_Corpus_roberta_predictions.csv")